<a target="_blank" href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/n4_bias_field_correct.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# N4 Bias field correction

The example below demonstrates extraction of radiomics features from CT scan and tumor segmentation for a batch of DICOM datasets.

### Install pyCERR

In [ ]:
%%capture
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR"

#### Import the required modules

In [1]:
from cerr import plan_container as pc
from cerr.dataclasses import scan as scn
import SimpleITK as sitk
import numpy as np

#### Load data into planC container

In [ ]:
# Load dicom to planC container
dcm_scan_dir = r"path/to/mr_dicom_dir"
planC = pc.loadDcmDir(dcm_scan_dir)

#### Apply N4BiasFieldCorrectionImageFilter from SimpleITK

In [4]:
# Apply N4 bias field correction filter
scanNum = 0
scanITKObj = planC.scan[scanNum].getSitkImage()
corrector = sitk.N4BiasFieldCorrectionImageFilter()
corrector.SetMaximumNumberOfIterations([50, 50, 50, 50])
outImg = corrector.Execute(scanITKObj)
logBiasFieldImg = corrector.GetLogBiasFieldAsImage(scanITKObj)
logBiasField3M = scn.getCERRScanArrayFromITK(logBiasFieldImg, scanNum, planC)
correctedImageFullResolution = planC.scan[scanNum].getScanArray() / np.exp(logBiasField3M)

#### Import the N4 corrected image into planC

In [5]:
# Add bias field corrected array to planC
x, y, z = planC.scan[scanNum].getScanXYZVals()
planC = pc.importScanArray(correctedImageFullResolution,x,y,z,'N4 Corrected',scanNum, planC)

### Visualize the original and N4 bias field corrected images

In [6]:
# Visualize original and bias field corrected images
from cerr import viewer as vwr
scanNum = [0,1]
strNum = [0]
doseNum = []
viewer, scan_layer, struct_layer, dose_layer, dvf_layer = \
    vwr.showNapari(planC, scan_nums=scanNum, struct_nums=strNum, dose_nums=doseNum, vectors_dict={}, displayMode='2d')
